In [ ]:
import time
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import StandardScaler


In [ ]:

# -------------------------------
# Define the Standard LSTM Model
# -------------------------------
class StandardLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1):
        super(StandardLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers)
        self.fc = nn.Linear(hidden_size, 1)
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(1), self.hidden_size, device=x.device)
        c0 = torch.zeros(self.num_layers, x.size(1), self.hidden_size, device=x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = out[-1]
        out = self.fc(out)
        return out

# -------------------------------
# Define the OPTM-LSTM Model and Cell
# -------------------------------
class OPTMLSTMCellTorch(nn.Module):
    def __init__(self, input_dim, hidden_size, gd_iters=7, gd_lr=0.0001):
        super(OPTMLSTMCellTorch, self).__init__()
        self.hidden_size = hidden_size
        self.gd_iters = gd_iters
        self.gd_lr = gd_lr
        self.linear = nn.Linear(input_dim, 4 * hidden_size, bias=True)
        self.recurrent_linear = nn.Linear(hidden_size, 4 * hidden_size, bias=False)
        self.sigmoid = torch.sigmoid
        self.tanh = torch.tanh

    def forward(self, x, h, c):
        x_features = x[:, :-1]
        guarantor = x[:, -1].view(-1, 1)
        
        z = self.linear(x_features) + self.recurrent_linear(h)
        z_i, z_f, z_c, z_o = z.chunk(4, dim=1)
        i = self.sigmoid(z_i)
        f = self.sigmoid(z_f)
        c_t = self.tanh(z_c)
        o = self.sigmoid(z_o)
        c_new = f * c + i * c_t
        h_temp = o * self.tanh(c_new)
        
        gated_vector = torch.cat([i, f, c_t, c_new, o, h_temp], dim=1)
        theta = torch.ones(6 * self.hidden_size, 1, device=x.device)
        for _ in range(self.gd_iters):
            y_pred = gated_vector @ theta
            error = y_pred - guarantor
            grad = (2 / x.size(0)) * (gated_vector.t() @ error)
            theta = theta - self.gd_lr * grad

        theta_parts = torch.chunk(theta, 6, dim=0)
        importance = [torch.mean(torch.abs(part)) for part in theta_parts]
        importance_stack = torch.stack(importance)
        max_idx = torch.argmax(importance_stack)

        if max_idx.item() == 0:
            new_h = i
        elif max_idx.item() == 1:
            new_h = f
        elif max_idx.item() == 2:
            new_h = c_t
        elif max_idx.item() == 3:
            new_h = c_new
        elif max_idx.item() == 4:
            new_h = o
        else:
            new_h = h_temp
        
        return new_h, c_new

class OPTMLSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(OPTMLSTM, self).__init__()
        self.hidden_size = hidden_size
        # Note: the cell input dim is input_size - 1 because the last feature is the guarantor.
        self.cell = OPTMLSTMCellTorch(input_dim=input_size - 1, hidden_size=hidden_size)
        self.fc = nn.Linear(hidden_size, 1)
        
    def forward(self, x):
        seq_len, batch, _ = x.shape
        device = x.device
        h = torch.zeros(batch, self.hidden_size, device=device)
        c = torch.zeros(batch, self.hidden_size, device=device)
        for t in range(seq_len):
            x_t = x[t]
            h, c = self.cell(x_t, h, c)
        out = self.fc(h)
        return out



In [ ]:
import time
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# -------------------------------
# Load Simulation Data
# -------------------------------
file_path = "datasets/test.csv"
data = pd.read_csv(file_path)

levels = 5
bid_price_cols = [f"bids[{i}].price" for i in range(levels)]
ask_price_cols = [f"asks[{i}].price" for i in range(levels)]
feature_columns = bid_price_cols + ask_price_cols

# Compute mid-price from the best bid and ask
data["mid_price"] = (data[bid_price_cols[0]] + data[ask_price_cols[0]]) / 2

# Ensure numeric types
data[feature_columns] = data[feature_columns].astype(float)
data["mid_price"] = data["mid_price"].astype(float)
data["mark_price"] = data["mark_price"].astype(float)

# Initialize scalers:
scaler_X = StandardScaler()
data[feature_columns] = scaler_X.fit_transform(data[feature_columns])

# Use separate scalers for target (mark_price) and guarantor (mid_price)
scaler_target = StandardScaler()
data["mark_price_scaled"] = scaler_target.fit_transform(data[["mark_price"]])

scaler_mid = StandardScaler()
data["mid_price_scaled"] = scaler_mid.fit_transform(data[["mid_price"]])

# Reset index for simulation streaming
simulation_data = data.reset_index(drop=True)

# -------------------------------
# Load Trained Models
# -------------------------------
input_size_combined = len(feature_columns) + 1
hidden_size = 64

# Assume StandardLSTM and OPTMLSTM classes are already defined
model_lstm = StandardLSTM(input_size_combined, hidden_size)
model_optm = OPTMLSTM(input_size_combined, hidden_size)

# Load saved weights (files should exist in the current directory)
model_lstm.load_state_dict(torch.load("standard_lstm_weights.pth", map_location="cpu"))
model_optm.load_state_dict(torch.load("optmlstm_weights.pth", map_location="cpu"))

# Set device to CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_lstm.to(device)
model_optm.to(device)

model_lstm.eval()
model_optm.eval()

# -------------------------------
# Simulation and Trading Setup (Unlimited Capital Injection)
# -------------------------------
portfolio_lstm = {"cash": 0.0, "asset": 0.0, "investment": 0.0}
portfolio_optm = {"cash": 0.0, "asset": 0.0, "investment": 0.0}

time_interval = 0.25  # 250 ms between LOB updates

# Grid trading strategy parameters
grid_spacing = 0.0005  # Relative threshold (0.05%)
abs_threshold = 2      # Absolute threshold (2 units)
buy_amount = 1000.0    # Fixed dollar amount to inject on each buy signal
sell_fraction = 0.1    # Fraction of held assets to sell on a sell signal
MIN_TRADE_UNITS = 0.01  # Minimum trade size (in asset units)

def execute_trade_grid_unlimited(portfolio, current_price, predicted_price, 
                                 grid_spacing=grid_spacing, abs_threshold=abs_threshold, 
                                 buy_amount=buy_amount, sell_fraction=sell_fraction, 
                                 min_trade_units=MIN_TRADE_UNITS):
    """
    Executes a grid trading strategy with unlimited capital.
    """
    action = "hold"
    rel_diff = predicted_price - current_price
    abs_diff = abs(rel_diff)
    
    # Buy signal
    if rel_diff > 0 and (predicted_price > current_price * (1 + grid_spacing) or abs_diff >= abs_threshold):
        units_bought = buy_amount / current_price
        if units_bought >= min_trade_units:
            portfolio["asset"] += units_bought
            portfolio["investment"] += buy_amount
            action = f"buy {units_bought:.2f} units (inject ${buy_amount:.2f})"
    # Sell signal
    elif rel_diff < 0 and (predicted_price < current_price * (1 - grid_spacing) or abs_diff >= abs_threshold):
        if portfolio["asset"] > 0:
            units_to_sell = portfolio["asset"] * sell_fraction
            if units_to_sell >= min_trade_units:
                sale_proceeds = units_to_sell * current_price
                portfolio["asset"] -= units_to_sell
                portfolio["cash"] += sale_proceeds
                action = f"sell {units_to_sell:.2f} units (receive ${sale_proceeds:.2f})"
    return action

results = []

# -------------------------------
# Simulation Loop
# -------------------------------
for idx, row in simulation_data.iterrows():
    time.sleep(time_interval)
    
    # Prepare the input vector: features + guarantor (scaled mid_price)
    features = row[feature_columns].to_numpy(dtype=np.float32)
    guarantor = np.array([float(row["mid_price_scaled"])], dtype=np.float32)
    input_vector = np.concatenate([features, guarantor])
    
    # Create tensor input with shape: (1, 1, input_size)
    input_tensor = torch.tensor(input_vector, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
    input_tensor = input_tensor.to(device)  # Move tensor to GPU
    
    with torch.no_grad():
        pred_lstm = model_lstm(input_tensor)
        pred_optm = model_optm(input_tensor)
    
    # Bring predictions back to CPU and convert back to original scale using scaler_target
    pred_lstm_original = scaler_target.inverse_transform(pred_lstm.cpu().numpy())
    pred_optm_original = scaler_target.inverse_transform(pred_optm.cpu().numpy())
    current_mark_original = scaler_target.inverse_transform(np.array([[float(row["mark_price_scaled"])]]))[0, 0]
    
    action_lstm = execute_trade_grid_unlimited(portfolio_lstm, current_mark_original, pred_lstm_original[0, 0])
    action_optm = execute_trade_grid_unlimited(portfolio_optm, current_mark_original, pred_optm_original[0, 0])
    
    value_lstm = portfolio_lstm["cash"] + portfolio_lstm["asset"] * current_mark_original
    value_optm = portfolio_optm["cash"] + portfolio_optm["asset"] * current_mark_original
    
    results.append({
        "step": idx,
        "current_mark": current_mark_original,
        "pred_lstm": pred_lstm_original[0, 0],
        "pred_optm": pred_optm_original[0, 0],
        "action_lstm": action_lstm,
        "action_optm": action_optm,
        "portfolio_value_lstm": value_lstm,
        "portfolio_value_optm": value_optm,
        "investment_lstm": portfolio_lstm["investment"],
        "investment_optm": portfolio_optm["investment"]
    })
    
    print(f"Step {idx}: Current Mark Price: {current_mark_original:.2f}, LSTM Pred: {pred_lstm_original[0, 0]:.2f}, OPTM Pred: {pred_optm_original[0, 0]:.2f}")
    print(f"         LSTM Action: {action_lstm}, OPTM Action: {action_optm}")
    print(f"         LSTM Portfolio Value: {value_lstm:.2f}, Investment: ${portfolio_lstm['investment']:.2f}")
    print(f"         OPTM Portfolio Value: {value_optm:.2f}, Investment: ${portfolio_optm['investment']:.2f}")

final_value_lstm = portfolio_lstm["cash"] + portfolio_lstm["asset"] * current_mark_original
final_value_optm = portfolio_optm["cash"] + portfolio_optm["asset"] * current_mark_original

profit_lstm = final_value_lstm - portfolio_lstm["investment"]
profit_optm = final_value_optm - portfolio_optm["investment"]

print("\nSimulation Completed.")
print(f"Standard LSTM Strategy Final Portfolio Value: ${final_value_lstm:.2f}, Investment: ${portfolio_lstm['investment']:.2f}, Profit: ${profit_lstm:.2f}")
print(f"OPTM-LSTM Strategy Final Portfolio Value: ${final_value_optm:.2f}, Investment: ${portfolio_optm['investment']:.2f}, Profit: ${profit_optm:.2f}")

steps = [r["step"] for r in results]
portfolio_vals_lstm = [r["portfolio_value_lstm"] for r in results]
portfolio_vals_optm = [r["portfolio_value_optm"] for r in results]

plt.figure(figsize=(10, 5))
plt.plot(steps, portfolio_vals_lstm, label="Standard LSTM Portfolio")
plt.plot(steps, portfolio_vals_optm, label="OPTM-LSTM Portfolio")
plt.xlabel("Simulation Step")
plt.ylabel("Portfolio Value ($)")
plt.title("Portfolio Value Evolution Over Simulation")
plt.legend()
plt.grid(True)
plt.show()
